In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 데이터 불러오기
- 데이터를 불러오도록 한다. 

In [ ]:
import pandas as pd 
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
print('Data Loading is done!')

In [ ]:
train.head()

In [ ]:
train.shape, test.shape

In [ ]:
print(train.info())

In [ ]:
print(test.info())

In [ ]:
print(test.head())

## EDA (Exploratory Data Analysis)
- Target 데이터에 대해 시각화를 해보도록 한다. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print(train['target'].value_counts())
news_class = train['target'].value_counts()
labels = ['Non-Disaster', 'Disaster']

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(labels, news_class, color=['green', 'orange'])

plt.show()

- Non Disaster 데이터와 Disaster 데이터를 비교해본다.
- 전체적으로 진짜 재난 뉴스의 데이터의 평균적인 길이가 길어보이는 경향이 있다. 

In [ ]:
disaster_tweet_len = train[train['target']==1]['text'].str.len()
non_disaster_tweet_len = train[train['target']==0]['text'].str.len()

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].hist(disaster_tweet_len, color='green')
ax[0].set_title("Disaster Tweet Length")

ax[1].hist(non_disaster_tweet_len, color='orange')
ax[1].set_title("Non Disaster Tweet Length")

fig.suptitle('All words in Tweets')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].boxplot(disaster_tweet_len, labels=['counts'], showmeans=True)
ax[0].set_title("Disaster Tweet Length")

ax[1].boxplot(non_disaster_tweet_len, labels=['counts'], showmeans=True)
ax[1].set_title("Non Disaster Tweet Length")

fig.suptitle('All words in Tweets')
plt.show()

In [ ]:
import numpy as np
disaster_tweet_len = train[train['target']==1]['text'].str.len()
non_disaster_tweet_len = train[train['target']==0]['text'].str.len()

print("Max Length of Disaster Tweet: {}".format(np.max(disaster_tweet_len)))
print("Min Length of Disaster Tweet: {}".format(np.min(disaster_tweet_len)))
print("Mean Length of Disaster Tweet: {:.2f}".format(np.mean(disaster_tweet_len)))
print("Median Length of Disaster Tweet: {}".format(np.median(disaster_tweet_len)))

print("Max Length of Non Disaster Tweet: {}".format(np.max(non_disaster_tweet_len)))
print("Min Length of Non Disaster Tweet: {}".format(np.min(non_disaster_tweet_len)))
print("Mean Length of Non Disaster Tweet: {:.2f}".format(np.mean(non_disaster_tweet_len)))
print("Median Length of Non Disaster Tweet: {}".format(np.median(non_disaster_tweet_len)))

- 실제 주요 키워드를 보면 차이점이 있어 보인다. 
    + Disaster Tweet: Outbreak, wreckage
    + Non Disaster Tweet:body%20bags, 

In [ ]:
from wordcloud import WordCloud, STOPWORDS

disaster_tweet_keywords = dict(train[train['target']==1]['keyword'].value_counts())
non_disaster_tweet_keywords = dict(train[train['target']==0]['keyword'].value_counts())

stopwords = set(STOPWORDS)
disaster_wordcloud = WordCloud(stopwords=stopwords, width=800, height=400, background_color="white").\
generate_from_frequencies(disaster_tweet_keywords)
non_disaster_wordcloud = WordCloud(stopwords=stopwords, width=800, height=400, background_color="white").\
generate_from_frequencies(non_disaster_tweet_keywords)


fig, ax = plt.subplots(1, 2, figsize=(16, 10))
ax[0].imshow(disaster_wordcloud, interpolation='bilinear')
ax[0].axis('off')
ax[0].set_title("Disaster Tweet")
ax[1].imshow(non_disaster_wordcloud, interpolation='bilinear')
ax[1].axis('off')
ax[1].set_title("Non Disaster Tweet")
fig.show()

## Feature Engineering

### 결측치 확인
- 텍스트에는 결측치가 존재하지 않음

In [ ]:
import pandas as pd
def check_na(data):
  isnull_na = (data.isnull().sum() / len(data)) * 100
  data_na = isnull_na.drop(isnull_na[isnull_na == 0].index).sort_values(ascending=False)
  missing_data = pd.DataFrame({'Missing Ratio': data_na, 
                               'Data Type': data.dtypes[data_na.index]})
  print("결측치 데이터 컬럼과 건수:\n", missing_data)

check_na(train)
check_na(test)

- 모델링에 필요 없는 변수들은 제거하였다. 

In [ ]:
test_id = test['id']

for datas in [train, test]:
  datas = datas.drop(['id', 'keyword', 'location'], axis=1, inplace=True)

train.shape, test.shape

### Text Cleansing
- HTML 태그 제거
- 특수문자 공백으로 바꾸기
- 대문자 소문자로 바꾼 후, 리스트로 만들기
- 불용어 제거하기

#### remove_url
- url만 제거하는 함수를 만든다. 

In [ ]:
import re

def remove_url(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'', text)

sample_text = "새로운 캐글 대회가 열렸습니다. 주소: https://www.kaggle.com/c/nlp-getting-started"
remove_url(sample_text)

#### remove_html
- 이번에는 html 코드를 제거하는 정규표현식을 확인하고, 실제 html 태그가 없어지는지 확인한다. 

In [ ]:
def remove_html(text):
  html = re.compile(r'<.*?>')
  return html.sub(r'', text)

sample_text ="""<div>
<h1> Real News or Fake News </h1>
<p> Kaggle Machine Learning </p>
</div>"""

print(remove_html(sample_text))

#### remove_emoji
- 이번에는 텍스트에 있는 emoji를 제거하는 코드를 작성한다.  


In [ ]:
!pip install emoji --upgrade

In [ ]:
import emoji
print(emoji.emojize('Phd is very easy!!! :thumbs_up:'))

In [ ]:
def remove_emoji(text):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

remove_emoji("Hello, 👍")

## Remove Special Letters
- 기존 특수 문자는 제거하는 함수를 작성한다. 

In [ ]:
def remove_punct(text):
  return re.sub("[^a-zA-Z]", " ", text)

sample_text = "Hello!, Can I have one question?.., Is it #Outbreak?"
remove_punct(sample_text)

#### 불용어 제거
- 불용어를 제거하기 위해서는 별도의 라이브러리가 필요하다. 

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

print("Total Length of stopwords:", len(stopwords.words('english')))
print(stopwords.words('english')[:10])

### data cleansing 
- 데이터 전처리를 전체 함수를 작성한다. 

In [ ]:
import string
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

def data_cleansing(text, remove_stopwords = False):
  # remove url 
  url = re.compile(r'https?://\S+|www\.\S+')
  cleaned_text = url.sub(r'', text)

  # remove html
  html = re.compile(r'<.*?>')
  cleaned_text = html.sub(r'', cleaned_text)

  # remove emoji
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  cleaned_text = emoji_pattern.sub(r'', cleaned_text)

  # Special Letters to empty space
  cleaned_text = re.sub("[^a-zA-Z]", " ", cleaned_text)

  # Lowercase
  cleaned_text = cleaned_text.lower().split()

  if remove_stopwords:
    stops = set(stopwords.words("english"))
    cleaned_text = [word for word in cleaned_text if not word in stops]
    clean_review = ' '.join(cleaned_text)
  else:
    clean_review = ' '.join(cleaned_text)

  return clean_review

- 이제 각 데이터에 반복문을 활용하여 텍스트 전처리를 시행한다. 
- 기존 text와 cleaned_text를 비교해본다. 

In [ ]:
clean_train_reviews = []
for datas in [train, test]:
    datas['cleaned_text'] = datas['text'].apply(lambda x : data_cleansing(x, remove_stopwords=True))

train.head(5)

In [ ]:
test.head(5)

## Text Transformation Vectorizer
- 텍스트는 문자이기 때문에, 이제 이를 숫자로 변경해야 한다. 

### Count Vectorizer
- CountVectorizer는 각 텍스트를 알파벳으로 정렬한 후, 빈도수로 계산한다. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['As you know, I want to be with you']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) 
print(vector.vocabulary_)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['As you know, I want to be with you', 
          'Thank you, but I cannot be with you']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) 
print(vector.vocabulary_)

### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ['Can I have lunch with you?', 
          'No, I cannot have it with you.', 
          'Because, I need to study later']
tfidfv = TfidfVectorizer().fit(corpus)
print(np.round(tfidfv.transform(corpus).toarray(), 2))
print(tfidfv.vocabulary_)

### Modeling

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer='char', sublinear_tf=True, ngram_range=(1, 3), max_features = 10000)
X = vectorizer.fit_transform(train['cleaned_text']).todense()
y = train['target'].values

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

X_testset = vectorizer.transform(test['cleaned_text']).todense()
print("The Shape of Test Dataset:", X_testset.shape)

y_test_pred = lgs.predict(X_testset)
print("The Predict Value:", y_test_pred)
y_test_pred = np.where(y_test_pred >= 0.5, 1, 0)
print("The Predict Class:", y_test_pred)

submission_file = pd.DataFrame({'id': test_id, 'target': y_test_pred})
print(submission_file.head())

submission_file.to_csv('submission.csv', index = False)